In [55]:
import requests
from bs4 import BeautifulSoup
import translators as ts
from tqdm.notebook import tqdm
import pandas as pd

ts.preaccelerate()

Using region Moscow server backend.
Preacceleration Process: 100%|██████████████████| 26/26 [01:27<00:00,  3.36s/it]


{'success': ['alibaba',
  'argos',
  'baidu',
  'bing',
  'caiyun',
  'deepl',
  'google',
  'iciba',
  'iflyrec',
  'itranslate',
  'lingvanex',
  'mglip',
  'modernMt',
  'myMemory',
  'papago',
  'qqFanyi',
  'qqTranSmart',
  'reverso',
  'sogou',
  'translateCom',
  'utibet',
  'youdao'],
 'fail': ['iflytek', 'niutrans', 'volcEngine', 'yandex']}

In [134]:
final_data = []
base = 'https://kaino.kotus.fi'
for page in tqdm(range(1, 46)):
    url = base + f'/algu/index.php?t=haku&o=hae&kieli=40&hakusana=&sivu={page}&tarkennettu=1&l=1'
    r = requests.get(url)
    r.encoding = r.apparent_encoding
    html = r.text

    soup = BeautifulSoup(html)
    words = soup.find_all('div', {'class': 'haku'})
    
    for word in words:
        links = word.find_all('a')
        to_word = [base + links[0]['href']] * 2
        bibs = [base + x['href'] for x in links[1:]]
        lexemes = [x.text for x in word.find_all('span', {'class': 'unicode'})]
        translations = [x.text for x in word.find_all('td', {'class': 'unicode2'})]
        final_data.extend(zip(lexemes, translations, to_word, bibs))

  0%|          | 0/45 [00:00<?, ?it/s]

In [80]:
df = pd.DataFrame(final_data, columns=['word', 'orig_trans', 'link_to_word', 'link_to_source'])

In [118]:
df['ru_trans'] = df['orig_trans'].apply(lambda x: ts.translate_text(x, from_language='fi-FI', to_language='ru-RU',
                                                                    translator='myMemory') if x else '')

In [129]:
def myMemory_translate(word, langpair):
    key = 'MY_MEMORY_TOKEN'
    r = requests.get('https://api.mymemory.translated.net/get',
                     params={
                         'q': word,
                         'langpair': langpair,
                         'key': key,
                         'de': 'MY_EMAIL',
                         'user': 'MY_EMAIL'
                     })
    return r.json()['responseData']['translatedText']

In [130]:
df['en_trans'] = df['orig_trans'].apply(lambda x: myMemory_translate(x, 'fi|en') if x else '')
df['ru_trans'] = df['orig_trans'].apply(lambda x: myMemory_translate(x, 'fi|ru') if x else '')

In [139]:
df = df[['word', 'ru_trans', 'en_trans', 'orig_trans', 'link_to_word', 'link_to_source']]

In [143]:
df[df.ru_trans != ''].to_excel('database_protosaami_filter.xlsx', index=False)